# Orchestrator Test - SearchWorkerPool MCP

Validate that the `Orchestrator` (SearchManager role) connects to `search_worker_pool` and executes search subtasks in parallel.

**Prerequisites**
- `OPENAI_API_KEY` (and Firecrawl key if required) present in `.env`
- Node 18+ with `npx firecrawl-mcp` reachable for the SearchAgent workers
- Python environment has `langchain`, `langchain-openai` packages installed

**What This Tests**
- Orchestrator initialization with `search_worker_pool` MCP
- `execute_subtasks` tool usage (provided by MCP pool)
- Parallel execution across SearchAgent workers
- Result aggregation


In [1]:
import os
import sys
from pathlib import Path

# get the current directory
cwd = os.getcwd()
print(f"Current working directory: {cwd}")

# REPO_ROOT = Path('/Users/leekayiu/Documents/workspace/manus_test/a2a_tree_backend')
REPO_ROOT = Path(cwd+'/../..')
if not REPO_ROOT.exists():
    raise FileNotFoundError(f'Repository path not found: {REPO_ROOT}')

SRC_PATH = REPO_ROOT / 'src'
if not SRC_PATH.exists():
    raise FileNotFoundError(f'Source path not found: {SRC_PATH}')

if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

env_path = REPO_ROOT / '.env'
if env_path.exists():
    try:
        from dotenv import load_dotenv
    except ImportError:
        load_dotenv = None

    if load_dotenv:
        load_dotenv(env_path)
    else:
        for line in env_path.read_text().splitlines():
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            key, _, value = line.partition('=')
            if key and value:
                os.environ.setdefault(key, value.strip().strip('"'))
else:
    print(f'.env not found at {env_path}')

if 'OPENAI_API_KEY' not in os.environ:
    raise RuntimeError('OPENAI_API_KEY must be set before running the agent.')


Current working directory: /Users/steven/search agent/search_agent_framework/examples/managers


In [2]:
import os
from langchain_openai import ChatOpenAI
from search_agent.orchestration import Orchestrator
import asyncio

MODEL_NAME = 'gpt-5.1'
TEST_SUBTASKS = (
"""Could you provide a detailed list of Michelin three-star restaurants in the whole world? I especially want to know the name, main cuisine style and exact address of each restaurant.

Please organize the results in one Markdown table with the following columns:
Restaurant, Main Cuisine Style, Address
Don't ask me any questions, just output the results according to the column without omitting cells arbitrarily. The output format is ```markdown
{data_content}
```"""
)

TEST_SUBTASKS = (
"""Could you provide all of Trump's public speeches after being president in 2025, including title, time, location, and key points. Don't ask me any questions, just provide the results."
"""
)

TEST_SUBTASKS = (
"""	
Please sort out all African countries involved in the "Belt and Road" initiative and their capitals as of June 2025, and query the surface area (in sq.km, using 2022 statistics, rounded to an integer), population density (people/sq.km of land area, using 2022 statistics, rounded to an integer), total population (in thousand, using 2023 statistics, rounded to an integer), and the proportion of merchandise trade in GDP (in percentage of GDP, using 2023 statistics, rounded to one decimal place) of these countries according to the statistics of the World Bank.

Please output the sorted data in the format of one Markdown table. The column names in the table are: Country, Capital City, Surface Area (km²), Population Density (people/km² of land area), Total Population (thousands), Merchandise Trade (% of GDP).
Don't ask me any questions, just output the results according to the columns without omitting cells arbitrarily. The output format is ```markdown
{data_content}
```
"""
)

TEST_SUBTASKS = (
"""I'd like to know the information about all Ford vehicles first launched and revived from January 1, 2010 to December 31, 2024 in the US (not including model updates or facelifts). Information is needed as follows: Model Name (Year), Price, Dimensions (L/W/H, mm), Wheelbase (mm), Maximum Torque (N·m), Front Suspension Type, Rear Suspension Type, Advanced Driving Assistance System (Please list all ADAS equipped in this model), Intelligent Parking Assist (Please list all IPA equipped in this model). All information is for the standard edition, and if it doesn't have ADAS or IPA, fill in '-'.

Please organize the results in one Markdown table with the following columns: Model Name, Price (USD, launch MSRPs), Dimensions L/W/H (mm), Wheelbase (mm), Max Torque (N·m), Front Suspension, Rear Suspension, Advanced-Driver-Assist Systems (ADAS²), Intelligent Parking Assist
Dimensions should adopt the data that exclude mirror and the launch MSRPs usually refers to price without destination fee.

Don't ask me any questions, just output the results according to the columns without omitting cells arbitrarily. The output format is ```markdown
{data_content}
```.
"""
)
# Create a query from TEST_SUBTASKS
# TARGET_QUERY = f"Please answer the following {len(TEST_SUBTASKS)} questions in parallel:\n"
# for i, subtask in enumerate(TEST_SUBTASKS, 1):
#     TARGET_QUERY += f"{i}. {subtask}\n"


# Create model
orchestrator_model = ChatOpenAI(
    model=MODEL_NAME,
    api_key=os.getenv('OPENAI_API_KEY'),
    base_url=os.getenv('OPENAI_BASE_URL')
)

# Create Orchestrator (connects to search_worker_pool)
orchestrator = Orchestrator(
    name="orchestrator",
    model=orchestrator_model,
    enable_sequential_thinking=True,
)
# Initialize Orchestrator (will connect to search_worker_pool)
await orchestrator.start()

print('Orchestrator initialized with search_worker_pool.')



Orchestrator initialized with search_worker_pool.


In [3]:
# Test 2: Streaming execution with stream()
print('='*60)
print('STREAMING UPDATES:')
print('='*60)
async for chunk in orchestrator.stream(TEST_SUBTASKS):
    print(chunk)
    print('-'*60)


STREAMING UPDATES:
{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 694, 'prompt_tokens': 2491, 'total_tokens': 3185, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5.1-2025-11-13', 'system_fingerprint': None, 'id': 'chatcmpl-D3hmWOUlR3PNYMRG0BQdEISQv6R0E', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--1e98d0b5-8a08-4f89-b0d1-cba49640d5b7-0', tool_calls=[{'name': 'execute_subtasks', 'args': {'subtasks': ['List all Ford brand vehicle nameplates that were first launched in the U.S. market or revived (a nameplate returning after being discontinued) between January 1, 2010 and December 31, 2024, excluding simple model-year updates or facelifts. Provide mode

In [4]:
# Cell 5: Cleanup
print("\nCleaning up...")
await orchestrator.close()
print('✓ Orchestrator and search_worker_pool closed.')



Cleaning up...
✓ Orchestrator and search_worker_pool closed.
